```text
┏━━━┓┏━━━┓┏━━━┓┏━━━┓┏━━━┓┏┓━━┏┓━━━━┏━━━┓┏┓━━━┏━━━┓┏━━━┓
┃┏━┓┃┃┏━┓┃┃┏━━┛┃┏━━┛┗┓┏┓┃┃┗┓┏┛┃━━━━┃┏━┓┃┃┃━━━┃┏━┓┃┃┏━┓┃
┃┃━┗┛┃┗━┛┃┃┗━━┓┃┗━━┓━┃┃┃┃┗┓┗┛┏┛━━━━┃┃━┃┃┃┃━━━┃┃━┗┛┃┃━┃┃
┃┃┏━┓┃┏┓┏┛┃┏━━┛┃┏━━┛━┃┃┃┃━┗┓┏┛━━━━━┃┗━┛┃┃┃━┏┓┃┃┏━┓┃┃━┃┃
┃┗┻━┃┃┃┃┗┓┃┗━━┓┃┗━━┓┏┛┗┛┃━━┃┃━━━━━━┃┏━┓┃┃┗━┛┃┃┗┻━┃┃┗━┛┃
┗━━━┛┗┛┗━┛┗━━━┛┗━━━┛┗━━━┛━━┗┛━━━━━━┗┛━┗┛┗━━━┛┗━━━┛┗━━━┛
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
```

This jupyter book contains the code for the greedy solution generator used in the genetic algorithm as our starting populations.

The greey algorithm creates fast and random solutions to the problem, while all solution are feasible they are not optimal. 

The number of iterations you set in here will determine how many solutions you have to start with in the genetic algorithm, so ensure you set this to a number higher than the number of solutions you want to start with in the genetic algorithm.

In [1]:
import os
import json 
from collections import defaultdict, Counter
import random
from math import ceil
import json

from modules import config
from modules.logger import loggers

In [2]:
# Set the dataset you want to use, being either "small-dataset" or "full-dataset"
dataset = "small-dataset"

# Set the number of iterations for the greedy algorithm, ensure you set atleast a few more than the number of solutions you want to start with in the genetic algorithm
num_iterations = 50

In [3]:
# Create a unique run ID based on the parameters, this is used to identify the run in the results
# Don't modify this as it sets the format for the genetic algorithm.

run_id = f"greedy_short_{dataset}_{config.RUN_ID}"

## Data import and Setup

In [4]:
# Set the output directory and paths for the scenario and results
# The scenario directory matches up to the dataset which is set above
# The results directory is set to the results folder
current_dir = os.getcwd()
scenario = os.path.join(current_dir, "..", "dataset", f"{dataset}")
results_dir = os.path.join(current_dir, "results")
output_dir = results_dir 

with open(os.path.join(scenario, 'radio_units_new.json')) as ru_new_file: ru_data_new = json.load(ru_new_file)
with open(os.path.join(scenario, 'radio_units_exist.json')) as ru_existing_file: ru_data_existing = json.load(ru_existing_file)
with open(os.path.join(scenario, 'distributed_units_new.json')) as du_new_file: du_data_new = json.load(du_new_file)
with open(os.path.join(scenario, 'distributed_units_exist.json')) as du_existing_file: du_data_existing = json.load(du_existing_file)
with open(os.path.join(scenario, 'centralised_units.json')) as cu_file: cu_data = json.load(cu_file)
with open(os.path.join(scenario, 'ru_du_path_new.json')) as ru_du_new_file: ru_du_new_paths = json.load(ru_du_new_file)
with open(os.path.join(scenario, 'ru_du_path_exist.json')) as ru_du_existing_file: ru_du_existing_paths = json.load(ru_du_existing_file)
with open(os.path.join(scenario, 'du_cu_path_new.json')) as du_cu_new_file: du_cu_paths_new = json.load(du_cu_new_file)
with open(os.path.join(scenario, 'du_cu_path_exist.json')) as du_cu_existing_file: du_cu_paths_existing = json.load(du_cu_existing_file)
with open(os.path.join(scenario, 'road_distances.json')) as road_file: road_distances = json.load(road_file)
with open(os.path.join(scenario, 'user_map.json')) as users_file: user_ru_mapping = json.load(users_file)

In [5]:
# Initialise all data
RUs_new = {ru['ru_name']: {'RC': ru['capacity_bandwidth']} for ru in ru_data_new['radio_units_new']}
RUs_existing = {ru['ru_name']: {'RC': ru['capacity_bandwidth']} for ru in ru_data_existing['radio_units_existing']}
CUs = {cu['cu_name']: {'CC': cu['capacity_bandwidth'], 'CP': cu['capacity_ports']} for cu in cu_data['centralised_units']}
DUs_new = {du['du_name']: {'DC': du['capacity_bandwidth'], 'DP': du['capacity_ports']} for du in du_data_new['distributed_units_new']}
DUs_existing = {du['du_name']: {'DC': du['capacity_bandwidth'], 'DP': du['capacity_ports']} for du in du_data_existing['distributed_units_existing']}

# Create dictionaries and sets for all devices
DUs = {**DUs_new, **DUs_existing}
RUs = {**RUs_new, **RUs_existing}
RU_names_new = list(RUs_new.keys())             # New RUs
RU_names_existing = list(RUs_existing.keys())   # Existing RUs
RU_names = list(RUs.keys())                     # All RUs (new + existing)
DU_names_new = list(DUs_new.keys())             # new DUs
DU_names_existing = list(DUs_existing.keys())   # existing DUs
DU_names = list(DUs.keys())                     # All DUs (new + existing)
CU_names = list(CUs.keys())                     # All CUs

# User to RU Mapping and adding coverage rate
ur_map = {user['user_id']: user['assigned_ru'] for user in user_ru_mapping}
user_ids = ur_map.keys()
total_users = len(user_ids)
UC = int(total_users * config.COVERAGE_THRESHOLD)
ur_rng = {u: [r for r in RU_names if r in ur_map[u]] for u in user_ids}

In [6]:
# Convert the road distances into a dictionary with both directions
road_distance_dict = {}
for entry in road_distances:
    road_distance_dict[(entry['from'], entry['to'])] = entry['length']
    road_distance_dict[(entry['to'], entry['from'])] = entry['length']

# Store the segments and their costs
KS = defaultdict(int)

def add_path_segments_to_costs(path, cost=config.KY_COST, device_to_node_cost=0):
    """
    Add path segments to the KS dictionary, setting cost for road segments and device-to-node segments.
    """
    for i in range(len(path) - 1):
        s = (path[i], path[i + 1])
        reverse_segment = (path[i + 1], path[i])

        if isinstance(path[i], str) or isinstance(path[i + 1], str):  # Device-to-node segment
            KS[s] = device_to_node_cost
            KS[reverse_segment] = device_to_node_cost
        elif s in road_distance_dict:
            KS[s] = road_distance_dict[s] * cost
        elif reverse_segment in road_distance_dict:
            KS[reverse_segment] = road_distance_dict[reverse_segment] * cost
            
# Process new paths first      
for conn in ru_du_new_paths:
    add_path_segments_to_costs(conn['path'], cost=config.KY_COST)
for conn in du_cu_paths_new:
    add_path_segments_to_costs(conn['path'], cost=config.KY_COST)
    
# Process paths between existing RUs and existing/new DUs
for conn in ru_du_existing_paths:
    ru_name = conn['ru_name']
    du_name = conn['du_name']
    path = conn['path']

    if du_name not in DU_names_existing:
        add_path_segments_to_costs(path, cost=config.KY_COST)
    else:
        add_path_segments_to_costs(path, cost=config.KM_COST)
        
for conn in du_cu_paths_existing: add_path_segments_to_costs(conn['path'], cost=config.KM_COST)
    
ru_du_paths = ru_du_new_paths + ru_du_existing_paths
du_cu_paths = du_cu_paths_new + du_cu_paths_existing    
    
# Add device-to-node segments
for ru_du in ru_du_paths:
    add_path_segments_to_costs([ru_du['ru_name'], ru_du['path'][0]], device_to_node_cost=0)
    add_path_segments_to_costs([ru_du['path'][-1], ru_du['du_name']], device_to_node_cost=0)

for du_cu in du_cu_paths:
    add_path_segments_to_costs([du_cu['du_name'], du_cu['path'][0]], device_to_node_cost=0)
    add_path_segments_to_costs([du_cu['path'][-1], du_cu['cu_name']], device_to_node_cost=0)

## Functions for the Greedy Algorithm

In [7]:
def validate_assignments(selected_RUs, ru_user_map, used_ru_capacity, total_users):
    total_assigned_users = sum(len(ru_user_map[ru]) for ru in selected_RUs)
    total_capacity = sum(used_ru_capacity[ru] for ru in selected_RUs)
    assert total_capacity == total_assigned_users * config.UM_VALUE, f"Total capacity ({total_capacity}) does not match assigned users ({total_assigned_users})."

In [8]:
def initial_assignment_of_users_to_RUs(RUs, ur_map, ur_rng, ru_user_map, unassigned_users, ru_multipliers, used_ru_capacity, selected_RUs):
    """Assign users to RUs based on the user RU mapping and RU capacity."""

    # Generate a random multiplier between 0.005 and 0.1
    multiplier = random.uniform(0.005, 0.05)

    # Calculate the upper limit for user assignment
    max_users = int(UC + UC * multiplier)

    # Track the total number of assigned users
    assigned_users = 0

    # Randomly decide the order of iteration: forward, backward, or shuffled
    order = random.choice(["forward", "backward"])
    if order == "forward":
        user_keys = list(ur_map.keys())
    elif order == "backward":
        user_keys = list(ur_map.keys())[::-1]
    else:  # Shuffled
        user_keys = list(ur_map.keys())
        random.shuffle(user_keys)

    loggers['RU_logger'].info(f"Processing order: {order}")

    for u in user_keys:
        if assigned_users >= max_users:  # Stop assigning if upper limit is reached
            loggers['RU_logger'].info(f"Coverage requirement reached: Assigned {assigned_users} users (target: {UC}-{max_users}).")
            break

        best_ru = None
        best_cost = float('inf')
        for ru in ur_rng[u]:
            for multiplier in range(1, config.MR_VALUE + 1):  # Try each multiplier up to MR
                if used_ru_capacity[ru] + config.UM_VALUE <= RUs[ru]['RC'] * multiplier:  # Check RU capacity with multiplier
                    utilisation = used_ru_capacity[ru] / RUs[ru]['RC']       # Calculate RU utilisation
                    cost = random.uniform(0, 0.2)                            # Random cost to break ties
                    score = utilisation + cost                               # Combine utilisation and cost
                    if score < best_cost:                                    # Check if the current RU is better than the best RU
                        best_cost = score
                        best_ru = ru

        if best_ru:
            ru_user_map[best_ru].add(u)        # Assign the user to the best RU
            unassigned_users.discard(u)        # Remove the user from the unassigned set
            used_ru_capacity[best_ru] += config.UM_VALUE  # Update the used capacity of the RU
            ru_multipliers[best_ru] = max(ru_multipliers[best_ru], (used_ru_capacity[best_ru] - 1) // RUs[best_ru]['RC'] + 1)
            assigned_users += 1                # Increment the assigned users count
            selected_RUs.add(best_ru)          # Add the RU to the selected RUs

        else:
            unassigned_users.add(u)  # Add the user to unassigned if no suitable RU was found

    # Validation: Check if the number of assigned users is within the desired range
    assert UC <= assigned_users <= max_users, \
        f"Assigned users ({assigned_users}) not within target range: {UC} to {max_users}."

    # Validate assignments after processing
    validate_assignments(selected_RUs, ru_user_map, used_ru_capacity, max_users)
    return ru_user_map, unassigned_users, ru_multipliers, used_ru_capacity

In [9]:
def minimise_total_RUs_with_reassignment(RUs, ru_user_map, used_ru_capacity, selected_RUs, ru_multipliers, assigned_users, unassigned_users):
    """Reassign users to RUs to minimise the total number of RUs used while ensuring coverage requirements are met."""

    for ru in sorted(ru_user_map.keys(), key=lambda r: (len(ru_user_map[r]) * config.UM_VALUE) / RUs[r]['RC'], reverse=True):
        loggers['RU_logger'].info(f"Processing RU {ru}, current users: {len(ru_user_map[ru])}")

        if len(assigned_users) >= UC:  # Check if required coverage is reached
            loggers['RU_logger'].info("minimise_total_RUs: Coverage requirement reached.")
            break

        for u in list(ru_user_map[ru]):  # Iterate over users assigned to this RU
            user_reassigned = False

            # Try reassigning the user to another RU
            for new_ru in ur_rng[u]:
                if new_ru == ru:  # Skip reassignment to the same RU
                    continue

                for multiplier in range(1, config.MR_VALUE + 1):
                    capacity = RUs[new_ru]['RC'] * multiplier
                    if used_ru_capacity[new_ru] + config.UM_VALUE <= capacity:
                        # Remove the user from the current RU
                        ru_user_map[ru].remove(u)
                        used_ru_capacity[ru] -= config.UM_VALUE

                        # Assign the user to the new RU
                        ru_user_map[new_ru].add(u)
                        used_ru_capacity[new_ru] += config.UM_VALUE
                        ru_multipliers[new_ru] = max(ru_multipliers[new_ru], (used_ru_capacity[new_ru] - 1) // RUs[new_ru]['RC'] + 1)
                        selected_RUs.add(new_ru)

                        loggers['RU_logger'].info(f"minimise_total_RUs: User {u} reassigned from RU {ru} to RU {new_ru}.")
                        user_reassigned = True
                        break

                if user_reassigned:
                    break

            # If the user could not be reassigned, leave them in the current RU
            if not user_reassigned:
                loggers['user_logger'].info(f"minimise_total_RUs: User {u} could not be reassigned and remains in RU {ru}.")

    # Recalculate `assigned_users` based on `ru_user_map`
    assigned_users = set()
    for ru_users in ru_user_map.values():
        assigned_users.update(ru_users)

    # Assign additional users if `UC` is not met
    if len(assigned_users) < UC:
        loggers['RU_logger'].info(f"Assigning unassigned users to meet coverage requirement UC={UC}.")
        for u in unassigned_users.copy():
            for ru in ur_rng[u]:
                for multiplier in range(1, config.MR_VALUE + 1):
                    if used_ru_capacity[ru] + config.UM_VALUE <= RUs[ru]['RC'] * multiplier:
                        # Assign the user to the RU
                        ru_user_map[ru].add(u)      # Update the RU user map
                        used_ru_capacity[ru] += config.UM_VALUE  # Update the used capacity of the RU
                        ru_multipliers[ru] = max(ru_multipliers[ru], (used_ru_capacity[ru] - 1) // RUs[ru]['RC'] + 1)
                        selected_RUs.add(ru)        # Add the RU to the selected RUs
                        assigned_users.add(u)       # Add the user to assigned users
                        unassigned_users.remove(u)  # Remove the user from unassigned users

                        loggers['RU_logger'].info(f"Assigned unassigned user {u} to RU {ru}.")
                        break
                if u not in unassigned_users:  # Stop if the user was assigned
                    break

    # Validation: Check consistency between `ru_user_map` and `used_ru_capacity`
    total_users_in_rus = sum(len(users) for users in ru_user_map.values())
    total_capacity_used = sum(used_ru_capacity.values()) // config.UM_VALUE

    loggers['logger'].info(f"Validation: Total users in RUs = {total_users_in_rus}, Total capacity used = {total_capacity_used}")
    assert total_users_in_rus == total_capacity_used, \
        f"Mismatch: Users in RUs ({total_users_in_rus}) != Used Capacity ({total_capacity_used})"
    assert len(assigned_users) >= UC, \
        f"Coverage requirement not met: Assigned users = {len(assigned_users)}, UC = {UC}"

    return selected_RUs, used_ru_capacity, list(assigned_users), ru_multipliers


In [10]:
def post_processing_of_RUs(RUs, ru_user_map, selected_RUs, used_ru_capacity, ru_multipliers, ur_rng):
    """
    Post-process the selected RUs by reassigning users to better utilise the RUs.
    The function returns the updated selected RUs, used RU capacity, and multipliers for each RU.
    """
    for current_ru in list(selected_RUs):
        for multiplier in range(1, config.MR_VALUE + 1):                                             # Check capacity with multipliers
            if used_ru_capacity[current_ru] <= (RUs[current_ru]['RC'] * multiplier):    # Check if RU is underutilised
                users_to_reassign = list(ru_user_map[current_ru])                       # Create a copy to iterate over
                reassigned = 0                                                          # Track the number of reassigned users
                for u in users_to_reassign:                                             # Iterate over users in the RU
                    reassigned_flag = False                                             # Track if the user is reassigned
                    for new_ru in ur_rng[u]:
                        if new_ru in selected_RUs and used_ru_capacity[new_ru] + config.UM_VALUE <= RUs[new_ru]['RC'] * multiplier:
                            if u in ru_user_map[current_ru]:        # Ensure the user exists in the current RU's user map
                                ru_user_map[current_ru].remove(u)   # Remove user from current_ru
                                used_ru_capacity[current_ru] -= config.UM_VALUE  # Update capacity for current RU
                                ru_multipliers[current_ru] = max(1, (used_ru_capacity[current_ru] - 1) // RUs[current_ru]['RC'] + 1)        
                                                        
                                ru_user_map[new_ru].add(u)          # Add user to new_ru                                
                                used_ru_capacity[new_ru] += config.UM_VALUE      # Update capacity for new RU
                                ru_multipliers[new_ru] = max(1, (used_ru_capacity[new_ru] - 1) // RUs[new_ru]['RC'] + 1)
                                loggers['user_logger'].info(f"post_processing: User {u} reassigned from RU {current_ru} to RU {new_ru}.")
                                loggers['RU_logger'].info(f"post_processing: Reassigned user {u} from RU {current_ru} to RU {new_ru}.")
                                reassigned += 1
                                reassigned_flag = True
                            break  # Stop checking other RUs once reassigned
                    if reassigned_flag:
                        break  # Break from user loop once reassigned
        # check if RU has no users left, or no capacity used and move ro unselected
        unselected_RUs = set()
        if used_ru_capacity[current_ru] == 0 or len(ru_user_map[current_ru]) == 0:
            loggers['RU_logger'].info(f"post_processing: RU {current_ru} has no users or capacity used.")
            unselected_RUs.add(current_ru)      # Add to unselected RUs
            selected_RUs.remove(current_ru)     # Remove from selected RUs
            del ru_user_map[current_ru]         # Remove RU from user map
            del used_ru_capacity[current_ru]    # Remove RU from used capacity
            del ru_multipliers[current_ru]      # Remove RU from multipliers
    return selected_RUs, used_ru_capacity, ru_multipliers

In [11]:
def assign_RUs_to_DUs(RUs, ru_user_map, ru_multipliers, DUs, fibre_connections_ru_du, used_du_capacity, used_du_ports, ru_du_paths, selected_DUs, ru_to_du_connections, selected_paths, selected_RUs, segment_usage_count, du_multipliers):
    """
    Assign RUs to DUs based on the RU capacity and DU capacity.
    Iterate over the selected RUs and assign them to the best DU based on the path cost.
    The function returns the updated RU to DU connections, DU capacity, fibre connections, and selected paths.
    """
    for ru in selected_RUs:
        best_du = None
        best_cost = float('inf')
        best_path = None
        required_fibres = ru_multipliers[ru] * ceil(RUs[ru]['RC'] / config.FC_VALUE) # Calculate required fibres for the current RU based on the RU multiplier

        for du in DUs:
            for multiplier in range(1, config.MD_VALUE + 1): # Try each multiplier up to MD
                total_capacity = used_du_capacity[du] + RUs[ru]['RC'] * ru_multipliers[ru] # Calculate total capacity for the DU
                total_ports = used_du_ports[du] + required_fibres                          # Calculate total ports for the DU
                loggers['DU_logger'].info(f"assign_RUs_to_DUs: DU {du}, capacity: {total_capacity}, ports: {total_ports}")

                # Check if the DU has enough capacity and ports
                if (total_ports <= DUs[du]['DP'] * multiplier and total_capacity <= DUs[du]['DC'] * multiplier):
                    for path in ru_du_paths:
                        if path['ru_name'] == ru and path['du_name'] == du:
                            # Calculate path cost considering only new segments
                            path_cost = sum(KS[(path['path'][i], path['path'][i + 1])] for i in range(len(path['path']) - 1) if (path['path'][i], path['path'][i + 1]) not in selected_paths)
                            loggers['DU_logger'].info(f"Segments used: {selected_paths}")
                            loggers['DU_logger'].info(f"assign_RUs_to_DUs: Path cost for RU {ru} to DU {du}: {path_cost}")

                            # Combine utilisation and cost to find the best DU
                            du_utilisation = total_capacity / (DUs[du]['DC'] * multiplier)
                            score = du_utilisation + path_cost
                            if score < best_cost: # Check if the current DU is better than the best DU
                                best_cost = score
                                best_du = du
                                best_path = path['path']

        if best_du:
            selected_DUs.add(best_du)                                       # Add the DU to the selected DUs
            ru_to_du_connections[ru] = best_du                              # Add the RU to DU connection
            used_du_capacity[best_du] += RUs[ru]['RC'] * ru_multipliers[ru] # Update the used capacity of the DU
            used_du_ports[best_du] += required_fibres                       # Update the used ports of the DU
            fibre_connections_ru_du[best_du] = fibre_connections_ru_du.get(best_du, 0) + required_fibres # Update the fibre connections for the DU
            du_multipliers[best_du] = max(1, (used_du_capacity[best_du] - 1) // DUs[best_du]['DC'] + 1, (used_du_ports[best_du] - 1) // DUs[best_du]['DP'] + 1) # Update the multiplier for the DU
            loggers['RU_logger'].info(f"assign_RUs_to_DUs: RU {ru} assigned to DU {best_du}.")
            loggers['DU_logger'].info(f"assign_RUs_to_DUs: Assigned RU {ru} to DU {best_du}.")
            loggers['DU_logger'].info(f"assign_RUs_to_DUs: Total assigned RUs: {len(ru_to_du_connections)}, capacity: {used_du_capacity[best_du]}, ports: {used_du_ports[best_du]}, multiplier: {du_multipliers[best_du]}")
            
            if best_path: # Add the path to the selected paths and update segment usage count
                for i in range(len(best_path) - 1):
                    segment = (best_path[i], best_path[i + 1])
                    reverse_segment = (segment[1], segment[0])
                    selected_paths.add(segment)               # Add forward direction
                    selected_paths.add(reverse_segment)       # Add reverse direction
                    segment_usage_count[segment] += 1         # Add forward direction
                    segment_usage_count[reverse_segment] += 1 # Add reverse direction

    return (ru_user_map, used_du_capacity, fibre_connections_ru_du, used_du_ports, ru_to_du_connections, selected_paths, segment_usage_count, du_multipliers)

In [12]:
def validate_RUs_after_assignment(selected_RUs, ru_to_du_connections, ru_du_paths):
    """Validate the RUs after assignment to ensure all selected RUs are connected to a DU. Returns a list of disconnected RUs."""
    disconnected_rus = []
    for ru in selected_RUs:
        if ru not in ru_to_du_connections: disconnected_rus.append(ru)
    return disconnected_rus

In [13]:
def reconnect_or_remove_disconnected_RUs(disconnected_rus, DUs, RUs, ru_multipliers, du_multipliers, used_du_capacity, used_du_ports, ru_du_paths, ru_to_du_connections, selected_paths, fibre_connections_ru_du, selected_RUs, segment_usage_count):
    """
    Reconnect or remove disconnected RUs by assigning them to the best DU based on the path cost.
    The function returns the updated RU to DU connections, DU capacity, fibre connections, and selected paths.
    """
    for ru in disconnected_rus:
        best_du = None
        best_cost = float('inf')
        best_path = None

        for d in DUs.keys():
            for multiplier in range(1, config.MD_VALUE + 1):  # Try each multiplier up to MD
                if (used_du_capacity[d] + RUs[ru]['RC'] * ru_multipliers[ru] <= DUs[d]['DC'] * multiplier and used_du_ports[d] + ru_multipliers[ru] <= DUs[d]['DP'] * multiplier): # Check DU capacity and ports
                    for path in ru_du_paths: # Iterate over the RU-DU paths
                        if path['ru_name'] == ru and path['du_name'] == d:
                            path_cost = sum(KS[(path['path'][i], path['path'][i + 1])] for i in range(len(path['path']) - 1))
                            du_utilisation = used_du_capacity[d] / (DUs[d]['DC'] * multiplier)
                            score = path_cost + du_utilisation

                            if score < best_cost: # Check if the current DU is better than the best DU
                                best_cost = score
                                best_du = d
                                best_path = path['path']
                                best_multiplier = multiplier

        if best_du:
            # Update all of the DU parameters
            ru_to_du_connections[ru] = best_du
            used_du_capacity[best_du] = sum(RUs[r]["RC"] * ru_multipliers[r] for r in ru_to_du_connections if ru_to_du_connections[r] == best_du)
            fibre_connections_ru_du[best_du] = sum(ru_multipliers[r] * ceil(RUs[r]["RC"] / config.FC_VALUE) for r in ru_to_du_connections if ru_to_du_connections[r] == best_du)
            used_du_ports[best_du] = fibre_connections_ru_du[best_du]
            du_multipliers[best_du] = max(du_multipliers[best_du], ceil(used_du_capacity[best_du] / DUs[best_du]['DC']), ceil(used_du_ports[best_du] / DUs[best_du]['DP']))

            # Add the path to the selected paths and update segment usage count
            if best_path:
                for i in range(len(best_path) - 1):
                    segment = (best_path[i], best_path[i + 1])
                    reverse_segment = (best_path[i + 1], best_path[i])
                    selected_paths.add(segment)
                    selected_paths.add(reverse_segment)
                    segment_usage_count[segment] += 1
                    segment_usage_count[reverse_segment] += 1
        else:
            # Move RU to unselected and reset its multiplier and fibre connections
            selected_RUs.discard(ru)
            unselected_RUs = set()
            unselected_RUs.add(ru)
            if ru in ru_to_du_connections:
                du = ru_to_du_connections[ru]
                del ru_to_du_connections[ru]

                # Update all of the DU parameters
                used_du_capacity[du] = sum(RUs[r]["RC"] * ru_multipliers[r] for r in ru_to_du_connections if ru_to_du_connections[r] == du)
                fibre_connections_ru_du[du] = sum(ru_multipliers[r] * ceil(RUs[r]["RC"] / config.FC_VALUE) for r in ru_to_du_connections if ru_to_du_connections[r] == du)
                used_du_ports[du] = fibre_connections_ru_du[du]
                du_multipliers[du] = max(du_multipliers[du], ceil(used_du_capacity[du] / DUs[du]['DC']), ceil(used_du_ports[du] / DUs[du]['DP']))

                # Remove the path from the selected paths and update segment usage count
                for path in ru_du_paths:
                    if path['ru_name'] == ru and path['du_name'] == du:
                        for i in range(len(path['path']) - 1):
                            segment = (path['path'][i], path['path'][i + 1])
                            reverse_segment = (path['path'][i + 1], path['path'][i])
                            selected_paths.discard(segment)
                            selected_paths.discard(reverse_segment)
                            segment_usage_count[segment] -= 1
                            segment_usage_count[reverse_segment] -= 1

    return ru_to_du_connections, used_du_capacity, used_du_ports, selected_paths, selected_RUs, segment_usage_count, du_multipliers


In [14]:
def assign_DUs_to_the_CU(selected_DUs, du_cu_paths, road_distance_dict, used_cu_capacity, used_cu_ports, CUs, selected_CUs, du_to_cu_connections, fibre_connections_du_cu, selected_paths, used_du_capacity, segment_usage_count):
    """
    Assign DUs to CUs based on the DU capacity and CU capacity.
    Iterate over the selected DUs and assign them to the best CU based on the path cost.
    The function returns the updated CU to DU connections, CU capacity, fibre connections, and selected
    """
    cu_name = CU_names[0]
    for d in selected_DUs:
        best_path = None
        shortest_distance = float('inf')

        if used_cu_capacity[cu_name] + DUs[d]['DC'] <= CUs[cu_name]['CC'] and used_cu_ports[cu_name] + 1 <= CUs[cu_name]['CP']:
            for path in du_cu_paths:
                if path['du_name'] == d and path['cu_name'] == cu_name:
                    path_distance = sum(road_distance_dict[(path['path'][i], path['path'][i+1])] for i in range(len(path['path']) - 1) if (path['path'][i], path['path'][i+1]) in road_distance_dict)

                    if path_distance < shortest_distance:
                        shortest_distance = path_distance
                        best_path = path['path']

        if best_path:
            selected_CUs.add(cu_name)                   # Add the CU to the selected CUs
            used_cu_capacity[cu_name] += DUs[d]['DC']   # Update the used capacity of the CU
            used_cu_ports[cu_name] += 1                 # Update the used ports of the CU
            du_to_cu_connections[d] = cu_name           # Add the DU to CU connection

            for i in range(len(best_path) - 1):         # Add the path to the selected paths and update segment usage count
                segment = (best_path[i], best_path[i+1])
                selected_paths.add(segment)
                segment_usage_count[segment] += 1
            
    for d in selected_DUs:
        best_cu = None
        best_cost = float('inf')
        best_path = None
        used_bandwidth = used_du_capacity.get(d, 0)  # Get the used bandwidth for the DU
        required_fibres = ceil(used_bandwidth / config.FC_VALUE)  # Calculate exact number of fibres

        for c in CUs.keys():
            # Check if the CU can handle the required fibres and bandwidth
            if (used_cu_ports[c] + required_fibres <= CUs[c]['CP'] and used_cu_capacity[c] + used_bandwidth <= CUs[c]['CC']):
                
                for path in du_cu_paths:
                    if path['du_name'] == d and path['cu_name'] == c:
                        # Calculate path cost based on road distances
                        path_cost = sum(road_distance_dict[(path['path'][i], path['path'][i + 1])] for i in range(len(path['path']) - 1) if (path['path'][i], path['path'][i + 1]) in road_distance_dict)
                        
                        # Find the CU with the minimum cost
                        if path_cost < best_cost:
                            best_cost = path_cost
                            best_cu = c
                            best_path = path['path']

        if best_cu:
            # Assign DU to CU
            selected_CUs.add(best_cu)
            used_cu_capacity[best_cu] += used_bandwidth
            used_cu_ports[best_cu] += required_fibres

            # Update fibre connections for this DU
            fibre_connections_du_cu[d] = required_fibres

            # Add the path to the selected paths
            if best_path:
                for i in range(len(best_path) - 1):
                    segment = (best_path[i], best_path[i + 1])
                    selected_paths.add(segment)
                    segment_usage_count[segment] += 1

    return selected_CUs, used_cu_capacity, used_cu_ports, du_to_cu_connections, fibre_connections_du_cu, selected_paths, segment_usage_count

In [15]:
def is_feasible(results, RUs, DUs, CUs, ur_map):
    try:
        # Validate total number of RUs (including cells) equals the number of fibres for RU to DU
        total_rus_count = sum(results["rus_with_cells"].values())  # sum of all cells across all RUs
        total_fibre_connections_ru_du = sum(results["fibre_connections_ru_du"].values())

        assert total_rus_count == total_fibre_connections_ru_du, (f"Total RUs (by cells) {total_rus_count} does not match the number of fibre connections from RU to DU {total_fibre_connections_ru_du}")

        # Validate total used DU bandwidth matches total RU used capacity
        total_ru_capacity = 0
        for r, used_capacity in results["used_ru_capacity"].items():
            num_cells = results["rus_with_cells"].get(r, 0)
            total_capacity = RUs[r]['RC'] * num_cells
            total_ru_capacity += total_capacity

        total_du_used_bandwidth = sum(results["used_du_capacity"].values())
        assert total_du_used_bandwidth == total_ru_capacity, f"Total DU bandwidth used {total_du_used_bandwidth} does not match total RU capacity used {total_ru_capacity}"

        # Validate user coverage
        total_users = len(results["ru_user_map"])
        assigned_users = len(results["assigned_users"])
        total_ru_capacity_used = sum(results["used_ru_capacity"].values())

        # Validate total RU capacity divided by UM equals the number of assigned users
        expected_users = total_ru_capacity_used / config.UM_VALUE
        assert assigned_users == expected_users, f"Total assigned users {assigned_users} does not match the expected value {expected_users}"

        # Ensure every assigned user is connected to an RU
        all_connected_users = set()
        for ru, users in results["ru_user_map"].items():
            all_connected_users.update(users)

        unconnected_users = set(results["assigned_users"]) - all_connected_users
        assert not unconnected_users, f"The following users are not connected to any RU: {unconnected_users}"

        # Check if each user is assigned to a valid RU
        for user_id in results["assigned_users"]:
            # Derive assigned RU from ru_user_map
            assigned_ru = next((ru for ru, assigned_users in results["ru_user_map"].items() if user_id in assigned_users), None)
            valid_rus = ur_map.get(user_id, [])  # Get the valid RUs for the user
            assert assigned_ru in valid_rus, f"User {user_id} is assigned to RU {assigned_ru}, which is not in their valid RU list {valid_rus}"

        print("All validations passed successfully.")
        return True
    except AssertionError as e:
        print(f"Validation failed: {e}")
        return False

## Greedy Algorithm Implementation

In [16]:
def greedy_algorithm(RUs, DUs, CUs, ur_map, ur_rng, road_distance_dict, ru_du_paths, du_cu_paths, total_users):
    assigned_users = []
    selected_RUs = set()
    selected_DUs = set()
    selected_CUs = set()
    ru_to_du_connections = {}
    du_to_cu_connections = {}
    used_ru_capacity = defaultdict(int)
    used_du_capacity = defaultdict(int)
    used_cu_capacity = defaultdict(int)
    used_du_ports = defaultdict(int)
    used_cu_ports = defaultdict(int)
    selected_paths = set()
    ru_user_map = defaultdict(set)
    unassigned_users = set(ur_map.keys())
    total_distance_used = 0
    ru_multipliers = defaultdict(lambda: 1)
    du_multipliers = defaultdict(lambda: 1)
    fibre_connections_ru_du = {}
    fibre_connections_du_cu = {}
    segment_usage_count = defaultdict(int)

    ru_user_map, unassigned_users, ru_multipliers, used_ru_capacity = initial_assignment_of_users_to_RUs(RUs, ur_map, ur_rng, ru_user_map, unassigned_users, ru_multipliers, used_ru_capacity, selected_RUs)
    selected_RUs, used_ru_capacity, assigned_users, ru_multipliers = minimise_total_RUs_with_reassignment(RUs, ru_user_map, used_ru_capacity, selected_RUs, ru_multipliers, assigned_users, unassigned_users)

    selected_RUs, used_ru_capacity, ru_multipliers = post_processing_of_RUs(RUs, ru_user_map, selected_RUs, used_ru_capacity, ru_multipliers, ur_rng)

    assign_RUs_to_DUs(RUs, ru_user_map, ru_multipliers, DUs, fibre_connections_ru_du, used_du_capacity, used_du_ports, ru_du_paths, selected_DUs, ru_to_du_connections, selected_paths, selected_RUs, segment_usage_count, du_multipliers)

    disconnected_rus = validate_RUs_after_assignment(selected_RUs, ru_to_du_connections, ru_du_paths)   

    ru_to_du_connections, used_du_capacity, used_du_ports, selected_paths, selected_RUs, segment_usage_count, du_multipliers = reconnect_or_remove_disconnected_RUs(disconnected_rus, DUs, RUs, ru_multipliers, du_multipliers, used_du_capacity, used_du_ports, ru_du_paths, ru_to_du_connections, selected_paths, fibre_connections_ru_du, selected_RUs, segment_usage_count)

    selected_CUs, used_cu_capacity, used_cu_ports, du_to_cu_connections, fibre_connections_du_cu, selected_paths, segment_usage_count = assign_DUs_to_the_CU(selected_DUs, du_cu_paths, road_distance_dict, used_cu_capacity, used_cu_ports, CUs, selected_CUs, du_to_cu_connections, fibre_connections_du_cu, selected_paths, used_du_capacity, segment_usage_count)

    # COST CALCULATIONS
    # =========================================================================
    total_cu_cost = sum(fibre_connections_du_cu[d] * config.KB_COST for d in du_to_cu_connections)
    coverage_percentage = (len(assigned_users) / total_users) * 100

    # Calculate total segment cost, ensuring each segment is counted only once
    def normalise_segment(segment):
        return tuple(map(str, segment))  # Convert all elements to strings

    # Update calculation with normalised segments
    total_segment_cost = round(sum(segment_usage_count[segment] * KS[segment] for segment in selected_paths if segment in KS and normalise_segment(segment)[0] <= normalise_segment(segment)[1]))

    total_distance_used = round(sum(road_distance_dict[segment] for segment in selected_paths if segment in road_distance_dict))
    
    # Count RUs and DUs by number of cells
    ru_cell_counts = Counter(ru_multipliers[r] for r in selected_RUs)
    du_cell_counts = Counter(du_multipliers[d] for d in selected_DUs)

    # Build RU → cell mapping
    rus_with_cells = {r: ru_multipliers[r] for r in selected_RUs}
    dus_with_cells = {d: du_multipliers[d] for d in selected_DUs}

    # --- RU Costs ---
    # Total RU cost scales with number of cells * KV_COST
    total_ru_cost = sum(num_cells * config.KV_COST for num_cells in rus_with_cells.values())

    # --- DU Costs ---
    total_du_cost_device = 0
    free_existing_du_used = set()

    for d, num_cells in dus_with_cells.items():
        if d in DU_names_existing:
            if d not in free_existing_du_used:
                free_existing_du_used.add(d)   # First one is free
                loggers['DU_logger'].info(f"DU {d} is the first free existing DU.")
            else:
                # Subsequent existing DUs: cost for first cell + additional cells
                total_du_cost_device += config.KW_COST + (num_cells - 1) * config.KX_COST
        elif d in DU_names_new:
            # New DUs: same cost structure
            total_du_cost_device += config.KW_COST + (num_cells - 1) * config.KX_COST

    total_du_cost_fibres = sum(fibre_connections_ru_du[d] * config.KL_COST for d in selected_DUs)
    total_du_cost = total_du_cost_device + total_du_cost_fibres
    total_cost = total_ru_cost + total_du_cost + total_cu_cost + total_segment_cost
    penalty_cost = 0
    penalty_cost += total_cost * 10

    return {
        "cost_of_RUs": total_ru_cost,
        "cost_of_DUs": total_du_cost,
        "cost_of_CUs": total_cu_cost,
        "total_segment_cost": round(total_segment_cost),
        "total_cost": total_cost,
        "total_distance_used": round(total_distance_used),
        "selected_RUs": selected_RUs,
        "not_selected_RUs": set(RUs.keys()) - selected_RUs,
        "selected_DUs": selected_DUs,
        "not_selected_DUs": set(DUs.keys()) - selected_DUs,
        "selected_CUs": selected_CUs,
        "ru_to_du_connections": ru_to_du_connections,
        "du_to_cu_connections": du_to_cu_connections,
        "fibre_connections_ru_du": fibre_connections_ru_du,
        "fibre_connections_du_cu": fibre_connections_du_cu,
        "used_ru_capacity": {r: used_ru_capacity[r] for r in selected_RUs},
        "used_du_capacity": {d: sum(RUs[r]["RC"] * ru_multipliers[r] for r in ru_to_du_connections if ru_to_du_connections[r] == d)for d in selected_DUs},
        "used_cu_capacity": {cu: used_cu_capacity[cu] for cu in selected_CUs},
        "used_du_ports": {d: fibre_connections_ru_du[d] for d in selected_DUs},
        "selected_paths": selected_paths,
        "coverage_percentage": coverage_percentage,
        "ru_user_map": ru_user_map,
        "unassigned_users": unassigned_users,
        "assigned_users": assigned_users,
        "penalty_cost": penalty_cost,
        "rus_with_cells": {r: ru_multipliers[r] for r in selected_RUs},
        "dus_with_cells": {d: du_multipliers[d] for d in selected_DUs},
        "ru_cell_counts": ru_cell_counts,
        "du_cell_counts": du_cell_counts,
        "segment_usage_count": segment_usage_count,
    }

## Save Results

In [17]:
def save_greedy_results(results, file_path):
    with open(file_path, 'w') as file:
        total_cost = (results["cost_of_RUs"] + results["cost_of_DUs"] + results["cost_of_CUs"] + results["total_segment_cost"])

        # Build RU cell count fields dynamically
        ru_counts = [str(results["ru_cell_counts"].get(i, 0)) for i in range(1, config.MR_VALUE + 1)]
        du_counts = [str(results["du_cell_counts"].get(i, 0)) for i in range(1, config.MD_VALUE + 1)]

        summary_line = (
            f'{results["coverage_percentage"]:.0f},'
            f'{len(results["selected_RUs"])},'
            f'{len(results["selected_DUs"])},'
            f'{results["cost_of_RUs"]},'
            f'{results["cost_of_DUs"]},'
            f'{results["cost_of_CUs"]},'
            f'{results["total_segment_cost"]},'
            f'{total_cost},'
            f'{results["total_distance_used"]},'
            f'{len(results["ru_user_map"])},'
            f'{sum(results["fibre_connections_ru_du"].values())},'
            f'{sum(results["fibre_connections_du_cu"].values())},'
            + ",".join(ru_counts) + ","
            + ",".join(du_counts) + ","
            + f'{config.UM_VALUE},{config.MR_VALUE},{config.CR_VALUE},{config.FC_VALUE},greedy'
        )

        # Build dynamic header
        ru_headers = [f'RU_Num{i}cell' for i in range(1, config.MR_VALUE + 1)]
        du_headers = [f'DU_Num{i}cell' for i in range(1, config.MD_VALUE + 1)]

        header_line = (
            'Coverage_Percentage,Num_RUs,Num_DUs,Cost_RUs,Cost_DUs,Cost_CUs,Cost_Paths,Total_Cost,Total_Distance,Total_Users,'
            'FibreRU_DU,FibreDU_CU,' +
            ",".join(ru_headers) + "," +
            ",".join(du_headers) + "," +
            "UM,MR,CR,FC,Type\n"
        )

        file.write(header_line)
        file.write(summary_line + "\n")

        file.write('\n\n\n\n')
        file.write('-' * 80 + '\n\n')
        file.write(f'Results when coverage low: {config.COVERAGE_THRESHOLD}, UM: {config.UM_VALUE}, FC: {config.FC_VALUE}, MR: {config.MR_VALUE}, CR: {config.CR_VALUE}\n\n')
        file.write(f"With the costs of RU(KV): {config.KV_COST}, DU(KW): {config.KW_COST}, Fibre RU-DU(KL): {config.KL_COST}, Fibre DU-CU(KB): {config.KB_COST}\n\n")
        file.write(f'RU Cost: {config.RU_COST}, RU Install: {config.RU_INSTALL_COST}, DU Cost: {config.DU_COST}, DU Install: {config.DU_INSTALL_COST}\n\n')
        file.write('-' * 80 + '\n\n')
        file.write('\nMODEL RESULTS\n\n')
        file.write(f'Cost of road paths: ${results["total_segment_cost"]:,}\n')
        file.write(f'Cost of RUs: ${results["cost_of_RUs"]:,}\n')
        file.write(f'Cost of DUs: ${results["cost_of_DUs"]:,}\n')
        file.write(f'CU Cost: ${results["cost_of_CUs"]}\n\n')

        file.write(f'TOTAL COST: ${total_cost:,.0f}\n')
        file.write(f'{results["total_segment_cost"]} + {results["cost_of_RUs"]} + {results["cost_of_DUs"]} + {results["cost_of_CUs"]} = ${total_cost:.0f}\n\n\n')
        
        file.write(f'Number of RUs: {len(results["selected_RUs"]):,}\n')
        file.write('RU Cell Distribution:\n')
        for i in range(1, config.MR_VALUE + 1):
            count = results["ru_cell_counts"].get(i, 0)
            file.write(f'Number of RUs with {i} cell{"s" if i > 1 else ""}: {count:,}\n')
        file.write("\n\n")

        file.write(f'Number of DUs: {len(results["selected_DUs"]):,}\n')
        file.write('DU Cell Distribution:\n')
        for i in range(1, config.MD_VALUE + 1):
            count = results["du_cell_counts"].get(i, 0)
            file.write(f'Number of DUs with {i} cell{"s" if i > 1 else ""}: {count:,}\n')
        file.write("\n\n")

        file.write(f'Total number of fibre connections from RU to DU: {sum(results["fibre_connections_ru_du"].values())}\n')
        file.write(f'Total number of fibre connections from DU to CU: {sum(results["fibre_connections_du_cu"].values())}\n\n')

        file.write('\n\nCOVERAGE\n')
        file.write(f'User Coverage: {results["coverage_percentage"]:.2f}%\n')
        file.write(f'Count of covered users: {len(results["assigned_users"])}\n')
        file.write(f"Count of unassigned users: {len(results['unassigned_users'])}\n")
        file.write(f'Total Users: {total_users}\n\n\n')
        
        file.write('DISTANCE\n')
        file.write(f'Total distance in m: {results["total_distance_used"]}m\n')
        total_distance_km = results["total_distance_used"] / 1000
        file.write(f'Total distance in km: {total_distance_km:.2f} km\n\n\n')
        
        file.write('-' * 80 + '\n\n')
        file.write('\nDEVICES SELECTED AND PATHS\n\n')
        file.write(f'selected_rus = {sorted(results["selected_RUs"])}\n')
        file.write(f'not_selected_rus = {sorted(results["not_selected_RUs"])}\n')
        file.write(f'selected_dus = {sorted(results["selected_DUs"])}\n')
        file.write(f'not_selected_dus = {sorted(results["not_selected_DUs"])}\n\n')
        ru_to_du_connections = [(ru, du) for ru, du in results["ru_to_du_connections"].items()]
        file.write(f'ru_to_du_connections = {sorted(ru_to_du_connections)}\n')
        du_to_cu_connections = [(du, cu) for du, cu in results["du_to_cu_connections"].items()]
        file.write(f'du_to_cu_connections = {sorted(du_to_cu_connections)}\n\n\n')

        file.write('-' * 80 + '\n\n')
        
        # Fibre Connections Overview
        file.write('\nFIBRE CONNECTIONS OVERVIEW\n')
        file.write('-' * 80 + '\n')
        file.write(f'{"DU":<8}{"Total RUs":<11}{"Connected RUs":<61}\n')
        file.write('-' * 80 + '\n')

        max_ru_width = 61
        indent = ' ' * 19

        total_rus_count = 0

        for d in results["selected_DUs"]:
            connected_rus = [r for r, du in results["ru_to_du_connections"].items() if du == d]
            total_rus = len(connected_rus)
            total_rus_count += total_rus
            connected_rus_str = ', '.join(connected_rus)
            lines = []
            while len(connected_rus_str) > max_ru_width:
                cut_index = connected_rus_str.rfind(',', 0, max_ru_width)
                lines.append(connected_rus_str[:cut_index])
                connected_rus_str = connected_rus_str[cut_index + 2:]
            lines.append(connected_rus_str)
            file.write(f'{d:<8}{total_rus:<11}{lines[0]:<61}\n')
            for line in lines[1:]:
                file.write(f'{indent}{line:<61}\n')

        file.write('-' * 80 + '\n')
        file.write(f'{"TOTALS:":<8}{total_rus_count:<11}\n')
        file.write('-' * 80 + '\n\n\n')

        # DU Usage and Capacity Overview (Fibre Connections and Ports)
        file.write('DU USAGE AND CAPACITY OVERVIEW (FIBRE CONNECTIONS AND PORTS):\n')
        file.write('-' * 80 + '\n')
        file.write(f'{"DU":<10}{"Used Ports":<18}{"Fibre RU-DU":<18}{"Fibre DU-CU":<18}{"Unused Ports":<18}\n')
        file.write('-' * 80 + '\n')

        total_du_ports = 0
        total_fibre_connections_ru_du = 0
        total_fibre_connections_du_cu = 0
        total_unused_ports = 0

        for d in results["selected_DUs"]:
            ports = results["used_du_ports"].get(d, 0)
            fibre_to_rus = results["fibre_connections_ru_du"].get(d, 0)
            fibre_to_cus = results["fibre_connections_du_cu"].get(d, 0)
            unused_ports = max(0, fibre_to_cus - ports)

            total_du_ports += ports
            total_fibre_connections_ru_du += fibre_to_rus
            total_fibre_connections_du_cu += fibre_to_cus
            total_unused_ports += unused_ports

            file.write(f'{d:<10}{ports:<18}{fibre_to_rus:<18}{fibre_to_cus:<18}{unused_ports:<18}\n')

        file.write('-' * 80 + '\n')
        file.write(f'{"TOTALS:":<10}{total_du_ports:<18}{total_fibre_connections_ru_du:<18}{total_fibre_connections_du_cu:<18}{total_unused_ports:<18}\n')
        file.write('-' * 80 + '\n\n\n')
        
        # DU Usage and Capacity Overview (Bandwidth)
        file.write('DU USAGE AND CAPACITY OVERVIEW (BANDWIDTH):\n')
        file.write('-' * 80 + '\n')
        file.write(f'{"DU":<10}{"Used BW":<18}{"Available BW":<18}{"Unused BW":<18}{"Num Cells":<10}\n')
        file.write('-' * 80 + '\n')

        total_du_capacity = 0
        total_available_bandwidth = 0
        total_unused_bandwidth = 0
        total_num_du_cells = 0

        for d, used_capacity in results["used_du_capacity"].items():
            num_cells = results["dus_with_cells"].get(d, 0)
            total_capacity = DUs[d]['DC'] * num_cells

            fibre_to_cus = results["fibre_connections_du_cu"].get(d, 0)
            available_bandwidth = fibre_to_cus * config.FC_VALUE
            unused_bandwidth = max(0, available_bandwidth - used_capacity)

            total_du_capacity += used_capacity
            total_available_bandwidth += available_bandwidth
            total_unused_bandwidth += unused_bandwidth
            total_num_du_cells += num_cells

            file.write(f'{d:<10}{used_capacity:<18}{available_bandwidth:<18}{unused_bandwidth:<18}{num_cells:<10}\n')

        file.write('-' * 80 + '\n')
        file.write(f'{"TOTALS:":<10}{total_du_capacity:<18}{total_available_bandwidth:<18}{total_unused_bandwidth:<18}{total_num_du_cells:<10}\n')
        file.write('-' * 80 + '\n\n\n')

        # RU Usage and Capacity Overview (Fibre Connections and Users)
        file.write('RU USAGE AND CAPACITY OVERVIEW (FIBRE CONNECTIONS AND USERS):\n')
        file.write('-' * 80 + '\n')
        file.write(f'{"RU":<10}{"Num Cells":<10}{"Used":<15}{"Available":<15}{"Unused":<14}{"Num Users":<15}\n')
        file.write('-' * 80 + '\n')

        total_ru_capacity_used = 0
        total_ru_capacity_available = 0
        total_ru_capacity_unused = 0
        total_num_users = 0
        total_num_ru_cells = 0

        for r, used_capacity in results["used_ru_capacity"].items():
            num_cells = results["rus_with_cells"].get(r, 0)
            total_capacity = RUs[r]['RC'] * num_cells

            unused_capacity = total_capacity - used_capacity
            num_users = used_capacity // config.UM_VALUE

            total_ru_capacity_used += used_capacity
            total_ru_capacity_available += total_capacity
            total_ru_capacity_unused += unused_capacity
            total_num_users += num_users
            total_num_ru_cells += num_cells

            file.write(f'{r:<10}{num_cells:<10}{used_capacity:<15}{total_capacity:<15}{unused_capacity:<14}{num_users:<15}\n')

        file.write('-' * 80 + '\n')
        file.write(f'{"TOTALS:":<10}{total_num_ru_cells:<10}{total_ru_capacity_used:<15}{total_ru_capacity_available:<15}{total_ru_capacity_unused:<14}{total_num_users:<15}\n')
        file.write('-' * 80 + '\n\n\n')
        
        file.write('\n\n\n' + '-' * 60 + '\n')
        file.write(f'User counts: \n')
        file.write('-' * 60 + '\n\n')
        file.write(f'Covered Users: {results["assigned_users"]}\n\n\n\n')
        file.write(f'Unassigned Users: {results["unassigned_users"]}\n')  

    print(f"Results have been saved to {file_path}")

In [18]:
class PythonStyleBooleanEncoder(json.JSONEncoder): 
    def encode(self, obj): return super().encode(obj).replace("true", "True").replace("false", "False")

def _build_snapshot(results, *, include_unselected=False, users_mode="all", include_selection_sets=False):
    selected_RUs, selected_DUs, selected_CUs = set(results["selected_RUs"]), set(results["selected_DUs"]), set(results["selected_CUs"])
    assigned_users, unassigned_users = set(results["assigned_users"]), set(results["unassigned_users"])
    ru_user_map, ru_to_du, du_to_cu = results["ru_user_map"], results["ru_to_cu_connections"] if "ru_to_cu_connections" in results else {}, results["du_to_cu_connections"]
    used_ru_capacity, used_du_capacity, used_cu_capacity = results["used_ru_capacity"], results["used_du_capacity"], results["used_cu_capacity"]
    used_du_ports = results.get("used_du_ports", {})
    fibre_ru_du, fibre_du_cu = results.get("fibre_connections_ru_du", {}), results.get("fibre_connections_du_cu", {})
    rus_with_cells, dus_with_cells = results.get("rus_with_cells", {}), results.get("dus_with_cells", {})
    not_selected_RUs, not_selected_DUs = set(results.get("not_selected_RUs", [])), set(results.get("not_selected_DUs", []))

    # Precompute per-site counts
    num_rus_per_site = {ru: rus_with_cells.get(ru, 0) for ru in used_ru_capacity.keys()}
    num_dus_per_site = {du: dus_with_cells.get(du, 0) for du in selected_DUs}

    # Invert DU->CU once: CU -> [DUs]
    cu_to_dus = defaultdict(list)
    for du, cu in du_to_cu.items(): cu_to_dus[cu].append(du)

    # RUs domain
    ru_domain = selected_RUs if not include_unselected else set(used_ru_capacity.keys()).union(not_selected_RUs)
    rus = [{
        "name": ru,
        "is_selected": ru in selected_RUs,
        "num_rus": num_rus_per_site.get(ru, 0) if ru in selected_RUs else 0,
        "connected_users": ([u for u in ru_user_map.get(ru, []) if u in assigned_users] if ru in selected_RUs else []),
        "connected_du": (results["ru_to_du_connections"].get(ru, None) if ru in selected_RUs else None),
        "used_capacity": (used_ru_capacity.get(ru, 0) if ru in selected_RUs else 0),
    } for ru in (selected_RUs if not include_unselected else ru_domain)]

    # DUs domain
    du_domain = selected_DUs if not include_unselected else set(used_du_capacity.keys()).union(not_selected_DUs)
    # Precompute RU->DU inverted: DU -> [RUs] (only once)
    du_to_rus = defaultdict(list)
    for ru, du in results["ru_to_du_connections"].items(): du_to_rus[du].append(ru)

    dus = [{
        "name": du,
        "is_selected": du in selected_DUs,
        "used_ports": (used_du_ports.get(du, 0) if du in selected_DUs else 0),
        "fibre_ru_du": (fibre_ru_du.get(du, 0) if du in selected_DUs else 0),
        "fibre_du_cu": (fibre_du_cu.get(du, 0) if du in selected_DUs else 0),
        "connected_rus": (du_to_rus.get(du, []) if du in selected_DUs else []),
        "connected_cu": (du_to_cu.get(du, None) if du in selected_DUs else None),
        "used_bandwidth": (used_du_capacity.get(du, 0) if du in selected_DUs else 0),
        "num_dus": (num_dus_per_site.get(du, 0) if du in selected_DUs else 0),
    } for du in (selected_DUs if not include_unselected else du_domain)]

    # CUs domain
    cus = [{
        "name": cu,
        "is_selected": cu in selected_CUs,
        "connected_dus": cu_to_dus.get(cu, []),
        "used_bandwidth": used_cu_capacity.get(cu, 0),
        "used_ports": len(cu_to_dus.get(cu, [])),
        "total_fibres": sum(fibre_du_cu.get(du, 0) for du in cu_to_dus.get(cu, [])),
    } for cu in selected_CUs]

    # Users: single-file includes assigned vs unassigned; iterations only assigned
    if users_mode == "all":
        user_ids = assigned_users.union(unassigned_users)
    else:
        user_ids = assigned_users

    # Precompute user -> RU once (inverse of ru_user_map)
    user_to_ru = {}
    for ru, users in ru_user_map.items():
        for u in users: user_to_ru[u] = ru

    users = [{
        "user_id": u,
        "assign_ru": (user_to_ru.get(u) if u in assigned_users else None),
        "is_assigned": u in assigned_users
    } for u in user_ids]

    # Paths & segments
    selected_paths = list(set(results["selected_paths"]))
    segment_usage_count = {str(k): v for k, v in results["segment_usage_count"].items()}

    snapshot = {
        "cu_info": cus,
        "du_info": dus,
        "ru_info": rus,
        "users": users,
        "total_cost": results["total_cost"],
        "cost_of_RUs": results["cost_of_RUs"],
        "cost_of_DUs": results["cost_of_DUs"],
        "cost_of_CUs": results["cost_of_CUs"],
        "total_segment_cost": results["total_segment_cost"],
        "total_distance_used": results["total_distance_used"],
        "coverage_percentage": results["coverage_percentage"],
        "selected_paths": selected_paths,
        "segment_usage_count": segment_usage_count,
    }

    if include_selection_sets:
        snapshot.update({
            "selected_RUs": list(selected_RUs),
            "selected_DUs": list(selected_DUs),
            "selected_CUs": list(selected_CUs),
            "ru_to_du_connections": results["ru_to_du_connections"],
            "du_to_cu_connections": results["du_to_cu_connections"],
            "assigned_users": list(assigned_users),
            "unassigned_users": list(unassigned_users),
        })
    return snapshot

def save_results_to_json(results, output_file_path):
    json_data = _build_snapshot(results, include_unselected=True, users_mode="all", include_selection_sets=False)
    with open(output_file_path, 'w') as f: json.dump(json_data, f, indent=4, cls=PythonStyleBooleanEncoder)
    print(f"Results successfully saved to {output_file_path}")

def save_all_iterations_to_json(results_list, output_file_path):
    json_data = {}
    for idx, results in enumerate(results_list):
        key = f"greedy{idx+1}"
        json_data[key] = _build_snapshot(results, include_unselected=False, users_mode="assigned", include_selection_sets=True)
    with open(output_file_path, 'w') as f: json.dump(json_data, f, indent=4, cls=PythonStyleBooleanEncoder)
    print(f"All iterations successfully saved to {output_file_path}")


In [19]:
def save_results(RUs, DUs, CUs, ur_map, ur_rng, road_distance_dict, ru_du_paths, du_cu_paths, total_users, output_file_path, best_json_output_path, all_json_output_path, header_text=None, num_iterations=10, logger=None):
    results_list = []
    best_cost = float('inf')
    best_iteration = None

    for i in range(num_iterations):
        # Run the greedy algorithm for the specified number of iterations
        results = greedy_algorithm(RUs, DUs, CUs, ur_map, ur_rng, road_distance_dict, ru_du_paths, du_cu_paths, total_users)

        # Check if the solution is feasible
        is_feasible_result = is_feasible(results, RUs, DUs, CUs, ur_map)
        print(f"Iteration {i + 1}: Feasibility check result = {is_feasible_result}")
        if not is_feasible_result:
            print(f"Iteration {i + 1}: Solution is not feasible and will be discarded.")
            continue

        results_list.append(results)

        total_cost = results['total_cost']
        total_distance = results['total_distance_used']
        num_rus = len(results['selected_RUs'])
        num_dus = len(results['selected_DUs'])
        num_users = len(results['assigned_users'])

        # Log iteration details
        print(f"Iteration {i + 1}: Total Cost={total_cost}, Total Distance={total_distance}, Num RUs={num_rus}, Num DUs={num_dus}, Num Users={num_users}, Coverage={results['coverage_percentage']:.2f}%, Cost of CUs={results['cost_of_CUs']}, Cost of DUs={results['cost_of_DUs']}, Cost of RUs={results['cost_of_RUs']}")
        
        log_data = {
            "iteration": i + 1,
            "total_cost": total_cost,
            "total_distance": total_distance,
            "num_RUs": num_rus,
            "num_DUs": num_dus,
            "num_users": num_users,
            "coverage_percentage": round(results['coverage_percentage']),
            "total_CU_cost": results['cost_of_CUs'],
            "total_DU_cost": results['cost_of_DUs'],
            "total_RU_cost": results['cost_of_RUs'],
        }

        # Ensure logger is functional
        if logger is not None and 'greedy_algo' in logger:
            print(f"Logging data for iteration {i + 1}: {log_data}")
            logger['greedy_algo'].debug(json.dumps(log_data))

        # Check if the current result is the best found so far
        if results['penalty_cost'] < best_cost:
            best_cost = results['penalty_cost']
            best_iteration = i + 1
            print(f"New best cost {best_cost} found at iteration {best_iteration}")

    if not results_list:
        print("No feasible solutions found in all iterations.")
        return []

    best_result = min(results_list, key=lambda x: x['total_cost'])
    save_greedy_results(best_result, output_file_path)
    save_results_to_json(best_result, best_json_output_path)
    save_all_iterations_to_json(results_list, all_json_output_path)
    print(f"Best result with cost {best_cost} found at iteration {best_iteration} saved to {output_file_path} and {best_json_output_path}")

    return results_list

## Call Greedy Algorithm

In [20]:
file_path = os.path.join(results_dir, f"{run_id}.txt")
best_json_file_path = os.path.join(results_dir, f"{run_id}_best.json")
all_json_file_path = os.path.join(results_dir, f"{run_id}_all.json")

greedy_results = save_results(RUs, DUs, CUs, ur_map, ur_rng, road_distance_dict, ru_du_paths, du_cu_paths, total_users, file_path, best_json_file_path, all_json_file_path, num_iterations=num_iterations, logger=loggers)

All validations passed successfully.
Iteration 1: Feasibility check result = True
Iteration 1: Total Cost=47995510, Total Distance=30252, Num RUs=34, Num DUs=5, Num Users=298, Coverage=97.70%, Cost of CUs=15200, Cost of DUs=72000, Cost of RUs=507650
Logging data for iteration 1: {'iteration': 1, 'total_cost': 47995510, 'total_distance': 30252, 'num_RUs': 34, 'num_DUs': 5, 'num_users': 298, 'coverage_percentage': 98, 'total_CU_cost': 15200, 'total_DU_cost': 72000, 'total_RU_cost': 507650}
New best cost 479955100 found at iteration 1
All validations passed successfully.
Iteration 2: Feasibility check result = True
Iteration 2: Total Cost=45049320, Total Distance=27414, Num RUs=30, Num DUs=5, Num Users=292, Coverage=95.74%, Cost of CUs=14400, Cost of DUs=70400, Cost of RUs=479050
Logging data for iteration 2: {'iteration': 2, 'total_cost': 45049320, 'total_distance': 27414, 'num_RUs': 30, 'num_DUs': 5, 'num_users': 292, 'coverage_percentage': 96, 'total_CU_cost': 14400, 'total_DU_cost': 7